# 1. Thay vì việc xài database của a Hoàng sau đó lại truy vấn từ database để mà lấy transcriptions làm input cho get Critierial 

=> thì thôi, làm 1 mạch luôn 

Hãy phân tích input và output của code này:

**INPUT:**
1. **Google Drive Folder ID:** 
```python
google_drive_files("1_oVkhMaU4M1-ZIkb5sL4XSRE1LmZUn3z")
```
- Đây là ID của một folder trên Google Drive
- Folder này được mong đợi có cấu trúc:
  - Một thư mục con "Audio" chứa các file âm thanh
  - Một thư mục con "Video" chứa các file video

2. **Credentials cho Google Drive API:**
```python
SCOPES = ['https://www.googleapis.com/auth/drive.metadata.readonly']
```
- Cần file `credentials.json` để xác thực với Google Drive API
- Sẽ tạo `token.pickle` để lưu token xác thực

3. **Audio Files:**
- File âm thanh được xử lý qua API tại endpoint:
```python
url = 'http://103.253.20.13:25029/role_assign'
```
- Yêu cầu secret key: 'codedongian'
- Yêu cầu parameter language (ví dụ: 'en' cho tiếng Anh)

**OUTPUT:**

1. **Cấu trúc thư mục local:**
```
audio_data/
    ├── [folder_name]/
    │   ├── Audio/         # Chứa file audio đã download
    │   ├── Text/          # Chứa file text được xử lý từ audio
    │   └── Transcript/    # Chứa transcript cuối cùng
```

2. **Database SQLite:**
- File: `audio_data.db`
- Cấu trúc bảng:
```sql
CREATE TABLE audio_data (
    id INTEGER PRIMARY KEY,
    file_name_video TEXT,
    file_name_audio TEXT, 
    url_video TEXT,
    url_audio TEXT,
    transcript TEXT
)
```

3. **Files được tạo ra:**
- Các file audio được download từ Google Drive vào thư mục `Audio/`
- File text chứa kết quả xử lý audio (`.txt`) trong thư mục `Text/`
  - Gồm 2 loại: `HV.txt` và `mentor.txt`
- File transcript cuối cùng trong thư mục `Transcript/`
  - Được tạo từ việc kết hợp `HV.txt` và `mentor.txt`
  - Format: `[thời gian] Speaker 1/2: nội dung`

**Quy trình xử lý:**

1. Download files từ Google Drive:
```python
url = f'https://drive.google.com/uc?id={file_id}'
gdown.download(url, local_path, quiet=False)
```

2. Xử lý audio thành text:
```python
output = process_audio(local_audio_file_path, 'en')
```

3. Kết hợp transcript từ 2 speakers:
```python
output = process_audio_outputs(output1, output2, new_filename)
```

4. Lưu thông tin vào database:
```python
insert_data(file_name_video, file_name_audio, url_video, url_audio, transcript, id)
```

5. Hiển thị dữ liệu đã xử lý:
```python
fetch_data(max_chars=10)  # Hiển thị preview của transcript với max 10 ký tự
```

**Ví dụ output hiển thị:**
```
ID: 1, FILE NAME VIDEO: video1.mp4, FILE NAME AUDIO: audio1.wav, 
URL VIDEO: https://drive.../view, URL AUDIO: https://drive.../view, 
TRANSCRIPT: [00:00:15] Speaker 1: Hello...
```

Đây là một pipeline xử lý audio hoàn chỉnh từ:
1. Download file từ Google Drive
2. Xử lý audio thành text 
3. Kết hợp transcript từ nhiều speakers
4. Lưu trữ vào database
5. Tổ chức file một cách có hệ thống

Bạn cần quan tâm nhất đến:
1. Folder ID input phải đúng cấu trúc
2. Cần có credentials cho Google Drive API
3. API xử lý audio phải hoạt động
4. Database SQLite phải có quyền write

Bỏ phần lưu output ở local, chỉ cần phần database, gồm có 2 trường thông tin:

1. URL_video (dạng như full này: https://drive.google.com/file/d/10qe6DkNX6up7-oG69HRG3B4j1A-WiI_o/view) 

Transcriptions 

Criteria:    là output khi       cho transcriptions cho qua API sau 

curl --location 'http://103.253.20.13:25034/analyze' \

--header 'Content-Type: application/json' \

--data '{"transcription": "[00:04:15] Mentee: Oh, good afternoon.\n[00:04:32] Mentor: Hello.\n[00:04:35] Mentor: Good afternoon."}'

Ah, để tôi giải thích rõ hơn:

1. **Về lưu tạm thời và xóa:**
Trong code hiện tại, file được xóa ngay sau khi xử lý xong (trong hàm `download_and_process_video`). Điều này có thể gây ra vấn đề nếu có lỗi xảy ra trong quá trình xử lý.

2. **Về database:**
Hiện tại chỉ lưu URL video và criteria, chưa lưu transcription.

Đây là phiên bản cải tiến:

```python
import os
import pickle
import json
import logging
import time
import shutil
from datetime import datetime, timedelta
from typing import Optional, Dict, Tuple
import requests
import gdown
from google.auth.transport.requests import Request
from google.oauth2.credentials import Credentials
from google_auth_oauthlib.flow import InstalledAppFlow
from googleapiclient.discovery import build
import sqlite3

# Configure logging
logging.basicConfig(
    level=logging.INFO,
    format='%(asctime)s - %(levelname)s - %(message)s'
)
logger = logging.getLogger(__name__)

class Config:
    """Configuration settings"""
    SCOPES = ['https://www.googleapis.com/auth/drive.metadata.readonly']
    TRANSCRIBE_API_URL = 'http://103.253.20.13:25029/role_assign'
    ANALYZE_API_URL = 'http://103.253.20.13:25034/analyze'
    DATABASE_PATH = 'videos.db'
    CREDENTIALS_PATH = 'credentials.json'
    TOKEN_PATH = 'token.pickle'
    SECRET_KEY = 'codedongian'
    TEMP_DIR = 'temp_videos'
    # Thời gian giữ file tạm (24 giờ)
    TEMP_FILE_TTL = timedelta(hours=24)

class DatabaseManager:
    """Handle database operations"""
    def __init__(self, db_path: str):
        self.db_path = db_path
        self.create_database()

    def create_database(self):
        """Create database with required table"""
        with sqlite3.connect(self.db_path) as conn:
            conn.execute('''
                CREATE TABLE IF NOT EXISTS videos (
                    id INTEGER PRIMARY KEY,
                    url_video TEXT NOT NULL,
                    transcription TEXT,
                    criteria TEXT,
                    processed_at TIMESTAMP DEFAULT CURRENT_TIMESTAMP
                )
            ''')

    def insert_video(self, url_video: str, transcription: str, criteria: str) -> bool:
        """Insert video data with timestamp"""
        try:
            with sqlite3.connect(self.db_path) as conn:
                conn.execute('''
                    INSERT INTO videos (url_video, transcription, criteria, processed_at)
                    VALUES (?, ?, ?, CURRENT_TIMESTAMP)
                ''', (url_video, transcription, criteria))
                return True
        except sqlite3.Error as e:
            logger.error(f"Database error: {e}")
            return False

class TempFileManager:
    """Manage temporary files"""
    def __init__(self, temp_dir: str, ttl: timedelta):
        self.temp_dir = temp_dir
        self.ttl = ttl
        os.makedirs(temp_dir, exist_ok=True)

    def cleanup_old_files(self):
        """Remove files older than TTL"""
        now = datetime.now()
        for filename in os.listdir(self.temp_dir):
            filepath = os.path.join(self.temp_dir, filename)
            file_modified = datetime.fromtimestamp(os.path.getmtime(filepath))
            if now - file_modified > self.ttl:
                try:
                    os.remove(filepath)
                    logger.info(f"Removed old temp file: {filepath}")
                except OSError as e:
                    logger.error(f"Error removing temp file: {e}")

    def cleanup_all(self):
        """Remove all temporary files and directory"""
        try:
            shutil.rmtree(self.temp_dir)
            logger.info("Cleaned up all temporary files")
        except OSError as e:
            logger.error(f"Error cleaning up temp files: {e}")

class TranscriptionProcessor:
    """Process audio transcription"""
    def __init__(self, config: Config):
        self.config = config

    def process_audio(self, audio_path: str, language: str = 'en') -> Optional[Dict]:
        """Process audio file using the role_assign API"""
        if not os.path.exists(audio_path):
            logger.error(f"Audio file not found: {audio_path}")
            return None

        with open(audio_path, 'rb') as audio_file:
            try:
                response = requests.post(
                    self.config.TRANSCRIBE_API_URL,
                    files={'audio': audio_file},
                    data={
                        'secret_key': self.config.SECRET_KEY,
                        'language': language
                    }
                )
                response.raise_for_status()
                return response.json()
            except requests.exceptions.RequestException as e:
                logger.error(f"Transcription API error: {e}")
                return None
            except json.JSONDecodeError as e:
                logger.error(f"JSON decode error: {e}")
                return None

    def analyze_transcript(self, transcript: str) -> Optional[str]:
        """Send transcript to analysis API and get criteria"""
        try:
            response = requests.post(
                self.config.ANALYZE_API_URL,
                headers={'Content-Type': 'application/json'},
                json={'transcription': transcript},
                timeout=30
            )
            response.raise_for_status()
            return response.json().get('criteria', '')
        except requests.exceptions.RequestException as e:
            logger.error(f"Analysis API error: {e}")
            return None

class VideoProcessor:
    """Process videos from Google Drive"""
    def __init__(self, config: Config):
        self.config = config
        self.db = DatabaseManager(config.DATABASE_PATH)
        self.temp_manager = TempFileManager(config.TEMP_DIR, config.TEMP_FILE_TTL)
        self.transcription_processor = TranscriptionProcessor(config)
        self.drive_service = self._authenticate_google_drive()

    def _authenticate_google_drive(self):
        """Authenticate with Google Drive API"""
        creds = None
        if os.path.exists(self.config.TOKEN_PATH):
            with open(self.config.TOKEN_PATH, 'rb') as token:
                creds = pickle.load(token)

        if not creds or not creds.valid:
            if creds and creds.expired and creds.refresh_token:
                creds.refresh(Request())
            else:
                flow = InstalledAppFlow.from_client_secrets_file(
                    self.config.CREDENTIALS_PATH, self.config.SCOPES)
                creds = flow.run_local_server(port=0)
            
            with open(self.config.TOKEN_PATH, 'wb') as token:
                pickle.dump(creds, token)

        return build('drive', 'v3', credentials=creds)

    def download_and_process_video(self, video_id: str, video_name: str) -> Tuple[Optional[str], Optional[str]]:
        """Download video and process its audio"""
        video_path = os.path.join(self.config.TEMP_DIR, video_name)
        
        try:
            # Download video
            download_url = f'https://drive.google.com/uc?id={video_id}'
            gdown.download(download_url, video_path, quiet=False)
            
            # Process audio
            transcription_result = self.transcription_processor.process_audio(video_path)
            
            if transcription_result:
                transcription = transcription_result.get('output', '')
                # Get criteria
                criteria = self.transcription_processor.analyze_transcript(transcription)
                return transcription, criteria
                
            return None, None
            
        except Exception as e:
            logger.error(f"Error processing video: {e}")
            return None, None

    def process_folder(self, folder_id: str):
        """Process videos in Google Drive folder"""
        try:
            # Cleanup old temporary files first
            self.temp_manager.cleanup_old_files()

            # Query for video files in the folder
            query = f"'{folder_id}' in parents"
            results = self.drive_service.files().list(
                q=query,
                fields="files(id, name)"
            ).execute()

            videos = results.get('files', [])
            
            for video in videos:
                # Generate video URL
                video_url = f"https://drive.google.com/file/d/{video['id']}/view"
                
                # Process video
                transcription, criteria = self.download_and_process_video(video['id'], video['name'])
                
                if transcription and criteria:
                    # Save to database
                    self.db.insert_video(video_url, transcription, criteria)
                    logger.info(f"Processed video: {video['name']}")
                else:
                    logger.error(f"Failed to process video: {video['name']}")

                # Sleep to avoid rate limiting
                time.sleep(3)

            # Cleanup all temporary files after processing
            self.temp_manager.cleanup_all()

        except Exception as e:
            logger.error(f"Error processing folder: {e}")
            # Ensure cleanup happens even if there's an error
            self.temp_manager.cleanup_all()

def main():
    """Main execution function"""
    config = Config()
    processor = VideoProcessor(config)
    
    try:
        # Process folder
        folder_id = "1_oVkhMaU4M1-ZIkb5sL4XSRE1LmZUn3z"  # Your folder ID
        processor.process_folder(folder_id)
        
        # Display results
        with sqlite3.connect(config.DATABASE_PATH) as conn:
            cursor = conn.execute('SELECT id, url_video, criteria, processed_at FROM videos')
            for row in cursor.fetchall():
                print(f"\nVideo ID: {row[0]}")
                print(f"URL: {row[1]}")
                print(f"Criteria: {row[2]}")
                print(f"Processed at: {row[3]}")
    
    except Exception as e:
        logger.error(f"Main execution error: {e}")
        raise

if __name__ == '__main__':
    main()

```

Những cải tiến chính:

1. **Quản lý file tạm tốt hơn:**
```python
class TempFileManager:
    # File tạm được giữ trong 24h 
    TEMP_FILE_TTL = timedelta(hours=24)
    
    def cleanup_old_files(self):
        # Xóa file cũ hơn 24h
    
    def cleanup_all(self):
        # Xóa tất cả file tạm sau khi xử lý xong
```

2. **Database schema mới:**
```sql
CREATE TABLE videos (
    id INTEGER PRIMARY KEY,
    url_video TEXT NOT NULL,
    transcription TEXT,        -- Thêm trường này
    criteria TEXT,
    processed_at TIMESTAMP     -- Thêm timestamp
)
```

3. **Quy trình xử lý:**
- Trước khi xử lý: Dọn file tạm cũ (>24h)
- Trong khi xử lý: Giữ file tạm để debug nếu cần
- Sau khi xử lý xong folder: Xóa tất cả file tạm
- Nếu có lỗi: Vẫn đảm bảo xóa file tạm

4. **Logging và Error Handling tốt hơn:**
- Log đầy đủ các bước xử lý
- Catch và handle exceptions ở mỗi bước
- Cleanup resources trong mọi trường hợp

Bạn thấy phiên bản này có ổn không? Tôi có thể điều chỉnh thêm nếu cần.

# 3. Code này đang lưu những trường nào nhỉ, 

nên check URL video từ Google Drive trong database. 

Nếu có rồi thì bỏ qua

Chưa có thì mới lấy 

Có phải bạn đang lấy URL_video từ Google Drive

Từ URL_video, tiến hành tìm 2 file audio, sử dụng 2 audio đó để cho qua : # Transcribe API url = 'http://103.253.20.13:25029/role_assign'

-> sau đó lấy: transcription

Sau đó từ transcription -> lấy được criteria ...

bằng cách cho qua # Analyze API url = 'http://103.253.20.13:25034/analyze'

Không đúng rồi. Code hiện tại đang làm sai quy trình. Hãy làm rõ quy trình đúng:

1. **Input từ Google Drive:**
- URL_video (dạng https://drive.google.com/file/d/{id}/view)
- Từ URL này sẽ có 2 file audio liên quan

2. **Quy trình xử lý:**
```mermaid
graph TD
    A[URL Video] --> B[Tìm 2 file audio liên quan]
    B --> C[Process Audio 1 qua role_assign API]
    B --> D[Process Audio 2 qua role_assign API]
    C --> E[Combine transcriptions]
    D --> E
    E --> F[Send to analyze API]
    F --> G[Get criteria]
```

Hãy sửa lại code cho đúng:

```python
import os
import pickle
import json
import logging
import time
from typing import Optional, Dict, Tuple, List
import requests
from google.auth.transport.requests import Request
from google.oauth2.credentials import Credentials
from google_auth_oauthlib.flow import InstalledAppFlow
from googleapiclient.discovery import build
import sqlite3

logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)

class Config:
    SCOPES = ['https://www.googleapis.com/auth/drive.metadata.readonly']
    TRANSCRIBE_API_URL = 'http://103.253.20.13:25029/role_assign'
    ANALYZE_API_URL = 'http://103.253.20.13:25034/analyze'
    DATABASE_PATH = 'videos.db'
    CREDENTIALS_PATH = 'credentials.json'
    TOKEN_PATH = 'token.pickle'
    SECRET_KEY = 'codedongian'

class DatabaseManager:
    def __init__(self, db_path: str):
        self.db_path = db_path
        self.create_database()

    def create_database(self):
        with sqlite3.connect(self.db_path) as conn:
            conn.execute('''
                CREATE TABLE IF NOT EXISTS videos (
                    id INTEGER PRIMARY KEY,
                    url_video TEXT NOT NULL UNIQUE,
                    transcription TEXT,
                    criteria TEXT,
                    created_at TIMESTAMP DEFAULT CURRENT_TIMESTAMP
                )
            ''')

    def url_exists(self, url_video: str) -> bool:
        try:
            with sqlite3.connect(self.db_path) as conn:
                cursor = conn.execute('SELECT COUNT(*) FROM videos WHERE url_video = ?', 
                                    (url_video,))
                return cursor.fetchone()[0] > 0
        except sqlite3.Error as e:
            logger.error(f"Database error checking URL: {e}")
            return False

    def insert_video(self, url_video: str, transcription: str, criteria: str) -> bool:
        try:
            with sqlite3.connect(self.db_path) as conn:
                conn.execute('''
                    INSERT INTO videos (url_video, transcription, criteria)
                    VALUES (?, ?, ?)
                ''', (url_video, transcription, criteria))
                return True
        except sqlite3.IntegrityError:
            logger.info(f"Video URL already exists: {url_video}")
            return False
        except sqlite3.Error as e:
            logger.error(f"Database error: {e}")
            return False

class AudioProcessor:
    def __init__(self, config: Config):
        self.config = config

    def process_audio(self, audio_file_path: str, language: str = 'en') -> Optional[Dict]:
        """Process single audio file through role_assign API"""
        if not os.path.exists(audio_file_path):
            logger.error(f"Audio file not found: {audio_file_path}")
            return None

        try:
            with open(audio_file_path, 'rb') as audio:
                response = requests.post(
                    self.config.TRANSCRIBE_API_URL,
                    files={'audio': audio},
                    data={
                        'secret_key': self.config.SECRET_KEY,
                        'language': language
                    }
                )
                response.raise_for_status()
                return response.json()
        except Exception as e:
            logger.error(f"Error processing audio: {e}")
            return None

    def combine_transcriptions(self, transcription1: Dict, transcription2: Dict) -> str:
        """Combine transcriptions from two audio files"""
        # Implement logic to combine transcriptions
        # This should match your specific format requirements
        combined = f"{transcription1['output']}\n{transcription2['output']}"
        return combined

class VideoProcessor:
    def __init__(self, config: Config):
        self.config = config
        self.db = DatabaseManager(config.DATABASE_PATH)
        self.audio_processor = AudioProcessor(config)
        self.drive_service = self._authenticate_google_drive()

    def _authenticate_google_drive(self):
        """Authenticate with Google Drive API"""
        creds = None
        if os.path.exists(self.config.TOKEN_PATH):
            with open(self.config.TOKEN_PATH, 'rb') as token:
                creds = pickle.load(token)

        if not creds or not creds.valid:
            if creds and creds.expired and creds.refresh_token:
                creds.refresh(Request())
            else:
                flow = InstalledAppFlow.from_client_secrets_file(
                    self.config.CREDENTIALS_PATH, self.config.SCOPES)
                creds = flow.run_local_server(port=0)
            with open(self.config.TOKEN_PATH, 'wb') as token:
                pickle.dump(creds, token)

        return build('drive', 'v3', credentials=creds)

    def find_related_audio_files(self, video_id: str) -> Tuple[Optional[str], Optional[str]]:
        """Find the two related audio files for a video"""
        try:
            # Implement logic to find related audio files
            # This should match your specific naming convention or relationship logic
            file_list = self.drive_service.files().list(
                q=f"'{video_id}' in parents",
                fields="files(id, name)"
            ).execute()

            audio_files = file_list.get('files', [])
            if len(audio_files) >= 2:
                return audio_files[0]['id'], audio_files[1]['id']
            return None, None
        except Exception as e:
            logger.error(f"Error finding audio files: {e}")
            return None, None

    def analyze_transcript(self, transcript: str) -> Optional[str]:
        """Get criteria from transcript using analyze API"""
        try:
            response = requests.post(
                self.config.ANALYZE_API_URL,
                headers={'Content-Type': 'application/json'},
                json={'transcription': transcript},
                timeout=30
            )
            response.raise_for_status()
            return response.json().get('criteria', '')
        except Exception as e:
            logger.error(f"Error analyzing transcript: {e}")
            return None

    def process_folder(self, folder_id: str):
        """Process videos in Google Drive folder"""
        try:
            query = f"'{folder_id}' in parents"
            results = self.drive_service.files().list(
                q=query,
                fields="files(id, name)"
            ).execute()

            videos = results.get('files', [])
            
            for video in videos:
                video_url = f"https://drive.google.com/file/d/{video['id']}/view"
                
                if self.db.url_exists(video_url):
                    logger.info(f"Skipping existing video: {video['name']}")
                    continue

                # Find related audio files
                audio1_id, audio2_id = self.find_related_audio_files(video['id'])
                if not audio1_id or not audio2_id:
                    logger.error(f"Could not find both audio files for video: {video['name']}")
                    continue

                # Process both audio files
                try:
                    # Download audio files to temp directory
                    os.makedirs('temp', exist_ok=True)
                    audio1_path = os.path.join('temp', f'audio1_{video["id"]}.wav')
                    audio2_path = os.path.join('temp', f'audio2_{video["id"]}.wav')
                    
                    # Process each audio file
                    transcription1 = self.audio_processor.process_audio(audio1_path)
                    transcription2 = self.audio_processor.process_audio(audio2_path)

                    if transcription1 and transcription2:
                        # Combine transcriptions
                        combined_transcript = self.audio_processor.combine_transcriptions(
                            transcription1, transcription2)
                        
                        # Get criteria
                        criteria = self.analyze_transcript(combined_transcript)
                        
                        if criteria:
                            # Save to database
                            self.db.insert_video(video_url, combined_transcript, criteria)
                            logger.info(f"Successfully processed video: {video['name']}")
                        else:
                            logger.error(f"Failed to get criteria for video: {video['name']}")
                    
                    # Cleanup
                    os.remove(audio1_path)
                    os.remove(audio2_path)
                
                except Exception as e:
                    logger.error(f"Error processing audio files: {e}")
                    continue

                time.sleep(3)  # Rate limiting

        except Exception as e:
            logger.error(f"Error processing folder: {e}")

def main():
    config = Config()
    processor = VideoProcessor(config)
    
    folder_id = "1_oVkhMaU4M1-ZIkb5sL4XSRE1LmZUn3z"
    processor.process_folder(folder_id)

if __name__ == '__main__':
    main()

```

Những thay đổi chính:

1. **Tách AudioProcessor riêng:**
- Xử lý từng file audio
- Combine transcriptions từ 2 file

2. **Thêm find_related_audio_files:**
```python
def find_related_audio_files(self, video_id: str) -> Tuple[Optional[str], Optional[str]]:
    # Tìm 2 file audio liên quan đến video
```

3. **Quy trình xử lý mới:**
```python
# 1. Lấy URL video
video_url = f"https://drive.google.com/file/d/{video['id']}/view"

# 2. Tìm 2 file audio liên quan
audio1_id, audio2_id = self.find_related_audio_files(video['id'])

# 3. Process từng file audio
transcription1 = self.audio_processor.process_audio(audio1_path)
transcription2 = self.audio_processor.process_audio(audio2_path)

# 4. Combine transcriptions
combined_transcript = self.audio_processor.combine_transcriptions(
    transcription1, transcription2)

# 5. Get criteria
criteria = self.analyze_transcript(combined_transcript)
```

Bạn cần:
1. Implement logic tìm 2 file audio liên quan trong `find_related_audio_files`
2. Implement logic combine transcriptions trong `combine_transcriptions`
theo đúng requirements của bạn.

Đây là quy trình đúng chứ?

Tôi sẽ viết code kết hợp và giải thích chi tiết từng bước:

```python
import os
import pickle
import json
import logging
import time
from typing import Optional, Dict, Tuple
import requests
import gdown
from google.auth.transport.requests import Request
from google.oauth2.credentials import Credentials
from google_auth_oauthlib.flow import InstalledAppFlow
from googleapiclient.discovery import build
import sqlite3
import re

# Configure logging
logging.basicConfig(
    level=logging.INFO,
    format='%(asctime)s - %(levelname)s - %(message)s'
)
logger = logging.getLogger(__name__)

class Config:
    """Configuration settings"""
    SCOPES = ['https://www.googleapis.com/auth/drive.metadata.readonly']
    TRANSCRIBE_API_URL = 'http://103.253.20.13:25029/role_assign'
    ANALYZE_API_URL = 'http://103.253.20.13:25034/analyze'
    DATABASE_PATH = 'videos.db'
    CREDENTIALS_PATH = 'credentials.json'
    TOKEN_PATH = 'token.pickle'
    SECRET_KEY = 'codedongian'
    TEMP_DIR = 'temp_files'

class DatabaseManager:
    """Handle database operations"""
    def __init__(self, db_path: str):
        self.db_path = db_path
        self.create_database()

    def create_database(self):
        """Create database with required table"""
        with sqlite3.connect(self.db_path) as conn:
            conn.execute('''
                CREATE TABLE IF NOT EXISTS videos (
                    id INTEGER PRIMARY KEY,
                    url_video TEXT NOT NULL UNIQUE,
                    transcription TEXT,
                    criteria TEXT,
                    created_at TIMESTAMP DEFAULT CURRENT_TIMESTAMP
                )
            ''')

    def url_exists(self, url_video: str) -> bool:
        """Check if URL already exists in database"""
        try:
            with sqlite3.connect(self.db_path) as conn:
                cursor = conn.execute('SELECT COUNT(*) FROM videos WHERE url_video = ?', 
                                    (url_video,))
                return cursor.fetchone()[0] > 0
        except sqlite3.Error as e:
            logger.error(f"Database error checking URL: {e}")
            return False

    def insert_video(self, url_video: str, transcription: str, criteria: str) -> bool:
        """Insert video data"""
        try:
            with sqlite3.connect(self.db_path) as conn:
                conn.execute('''
                    INSERT INTO videos (url_video, transcription, criteria)
                    VALUES (?, ?, ?)
                ''', (url_video, transcription, criteria))
                return True
        except sqlite3.IntegrityError:
            logger.info(f"Video URL already exists: {url_video}")
            return False
        except sqlite3.Error as e:
            logger.error(f"Database error: {e}")
            return False

class AudioProcessor:
    """Process audio files and transcriptions"""
    def __init__(self, config: Config):
        self.config = config
        os.makedirs(config.TEMP_DIR, exist_ok=True)

    def extract_dicts(self, s: str) -> list:
        """Extract dictionaries from JSON string"""
        pattern = r"\{[^{}]*\}"
        matches = re.findall(pattern, s)
        result = []
        for match in matches:
            try:
                match = match.replace("'", '"')
                d = json.loads(match)
                result.append(d)
            except json.JSONDecodeError:
                pass
        return result

    def format_time(self, milliseconds: int) -> str:
        """Format milliseconds to time string"""
        seconds = milliseconds // 1000
        minutes = seconds // 60
        hours = minutes // 60
        return f"{hours:02d}:{minutes%60:02d}:{seconds%60:02d}"

    def process_audio(self, audio_path: str, language: str = 'en') -> Optional[Dict]:
        """Process single audio file through API"""
        if not os.path.exists(audio_path):
            logger.error(f"Audio file not found: {audio_path}")
            return None

        with open(audio_path, 'rb') as audio_file:
            try:
                response = requests.post(
                    self.config.TRANSCRIBE_API_URL,
                    files={'audio': audio_file},
                    data={
                        'secret_key': self.config.SECRET_KEY,
                        'language': language
                    }
                )
                response.raise_for_status()
                return response.json()
            except Exception as e:
                logger.error(f"Error processing audio: {e}")
                return None

    def combine_transcriptions(self, hv_output: Dict, mentor_output: Dict) -> str:
        """Combine transcriptions from HV and mentor"""
        try:
            # Extract and format transcriptions
            hv_data = self.extract_dicts(hv_output['output'])
            mentor_data = self.extract_dicts(mentor_output['output'])

            # Mark speakers
            for item in hv_data:
                item['speaker'] = '1'
            for item in mentor_data:
                item['speaker'] = '2'

            # Combine and sort by time
            combined = sorted(hv_data + mentor_data, key=lambda x: x['start_time'])

            # Format output
            formatted = []
            for item in combined:
                time = self.format_time(item['start_time'])
                formatted.append(f"[{time}] Speaker {item['speaker']}: {item['text']}")

            return "\n".join(formatted)

        except Exception as e:
            logger.error(f"Error combining transcriptions: {e}")
            return ""

    def analyze_transcript(self, transcript: str) -> Optional[str]:
        """Get criteria from transcript"""
        try:
            response = requests.post(
                self.config.ANALYZE_API_URL,
                headers={'Content-Type': 'application/json'},
                json={'transcription': transcript},
                timeout=30
            )
            response.raise_for_status()
            return response.json().get('criteria', '')
        except Exception as e:
            logger.error(f"Error analyzing transcript: {e}")
            return None

class VideoProcessor:
    """Main processor for handling videos and their related files"""
    def __init__(self, config: Config):
        self.config = config
        self.db = DatabaseManager(config.DATABASE_PATH)
        self.audio_processor = AudioProcessor(config)
        self.drive_service = self._authenticate_google_drive()

    def _authenticate_google_drive(self):
        """Set up Google Drive authentication"""
        creds = None
        if os.path.exists(self.config.TOKEN_PATH):
            with open(self.config.TOKEN_PATH, 'rb') as token:
                creds = pickle.load(token)

        if not creds or not creds.valid:
            if creds and creds.expired and creds.refresh_token:
                creds.refresh(Request())
            else:
                flow = InstalledAppFlow.from_client_secrets_file(
                    self.config.CREDENTIALS_PATH, self.config.SCOPES)
                creds = flow.run_local_server(port=0)
            with open(self.config.TOKEN_PATH, 'wb') as token:
                pickle.dump(creds, token)

        return build('drive', 'v3', credentials=creds)

    def find_folders(self, folder_id: str) -> Tuple[Optional[str], Optional[str]]:
        """Find Audio and Video folders"""
        query = f"'{folder_id}' in parents and mimeType = 'application/vnd.google-apps.folder'"
        results = self.drive_service.files().list(q=query, fields="files(id, name)").execute()
        
        audio_folder = None
        video_folder = None
        
        for item in results.get('files', []):
            if item['name'] == 'Audio':
                audio_folder = item['id']
            elif item['name'] == 'Video':
                video_folder = item['id']
        
        return audio_folder, video_folder

    def process_folder(self, folder_id: str):
        """Main processing function"""
        try:
            # 1. Find Audio and Video folders
            audio_folder, video_folder = self.find_folders(folder_id)
            if not audio_folder or not video_folder:
                logger.error("Could not find both Audio and Video folders")
                return

            # 2. Get videos
            videos = self.drive_service.files().list(
                q=f"'{video_folder}' in parents",
                fields="files(id, name)"
            ).execute().get('files', [])

            # 3. Process each video
            for video in videos:
                video_url = f"https://drive.google.com/file/d/{video['id']}/view"
                
                # Skip if already processed
                if self.db.url_exists(video_url):
                    logger.info(f"Skipping existing video: {video['name']}")
                    continue

                # 4. Find corresponding audio files
                base_name = os.path.splitext(video['name'])[0]
                audio_files = self.drive_service.files().list(
                    q=f"'{audio_folder}' in parents and (name contains '{base_name}')",
                    fields="files(id, name)"
                ).execute().get('files', [])

                hv_file = None
                mentor_file = None
                for file in audio_files:
                    if 'HV' in file['name']:
                        hv_file = file
                    elif 'mentor' in file['name']:
                        mentor_file = file

                if not (hv_file and mentor_file):
                    logger.error(f"Missing audio files for video: {video['name']}")
                    continue

                # 5. Process audio files
                try:
                    # Download and process HV audio
                    hv_path = os.path.join(self.config.TEMP_DIR, f"hv_{video['id']}.wav")
                    gdown.download(f"https://drive.google.com/uc?id={hv_file['id']}", 
                                 hv_path, quiet=False)
                    hv_result = self.audio_processor.process_audio(hv_path)

                    # Download and process mentor audio
                    mentor_path = os.path.join(self.config.TEMP_DIR, f"mentor_{video['id']}.wav")
                    gdown.download(f"https://drive.google.com/uc?id={mentor_file['id']}", 
                                 mentor_path, quiet=False)
                    mentor_result = self.audio_processor.process_audio(mentor_path)

                    if hv_result and mentor_result:
                        # 6. Combine transcriptions
                        transcript = self.audio_processor.combine_transcriptions(
                            hv_result, mentor_result)
                        
                        # 7. Get criteria
                        criteria = self.audio_processor.analyze_transcript(transcript)
                        
                        if criteria:
                            # 8. Save to database
                            self.db.insert_video(video_url, transcript, criteria)
                            logger.info(f"Successfully processed video: {video['name']}")

                    # Cleanup temp files
                    os.remove(hv_path)
                    os.remove(mentor_path)

                except Exception as e:
                    logger.error(f"Error processing files for video {video['name']}: {e}")
                    continue

                time.sleep(3)  # Rate limiting

        except Exception as e:
            logger.error(f"Error in main processing: {e}")

def main():
    """Main execution function"""
    config = Config()
    processor = VideoProcessor(config)
    
    folder_id = "1_oVkhMaU4M1-ZIkb5sL4XSRE1LmZUn3z"
    processor.process_folder(folder_id)

if __name__ == '__main__':
    main()

```

**Quy trình xử lý:**

1. **Setup và Cấu hình:**
```python
class Config:
    # Lưu các thông số cấu hình: API URLs, paths, credentials
```

2. **Quản lý Database:**
```python
class DatabaseManager:
    # Tạo table videos(id, url_video, transcription, criteria, created_at)
    # Check URL tồn tại
    # Insert dữ liệu mới
```

3. **Xử lý Audio:**
```python
class AudioProcessor:
    # Process từng file audio qua API role_assign
    # Combine transcriptions từ 2 file HV và mentor
    # Analyze transcript để lấy criteria
```

4. **Main Processor:**
```python
class VideoProcessor:
    # Bước 1: Tìm folders Audio và Video
    # Bước 2: Lấy danh sách videos
    # Bước 3: Với mỗi video:
        # - Check URL đã tồn tại chưa
        # - Tìm 2 file audio tương ứng (HV và mentor)
        # - Download và process từng file audio
        # - Combine transcriptions
        # - Get criteria
        # - Lưu vào database
        # - Cleanup files tạm
```

**Cải tiến so với code cũ:**

1. **Cấu trúc rõ ràng hơn:**
- Tách thành các class riêng biệt
- Mỗi class có nhiệm vụ cụ thể
- Dễ maintain và debug

2. **Error handling tốt hơn:**
- Sử dụng logging thay vì print
- Try-except chi tiết
- Thông báo lỗi rõ ràng

3. **Cleanup tốt hơn:**
- Xóa files tạm sau khi xử lý
- Không tạo nhiều thư mục con

4. **Database tối ưu hơn:**
- Thêm UNIQUE constraint cho URL
- Thêm timestamp
- Lưu cả transcript và criteria

5. **Hiệu quả hơn:**
- Check trùng lặp trước khi xử lý
- Rate limiting để tránh lỗi API
- Quản lý resources tốt hơn

Bạn thấy cách tổ chức này thế nào?

# 5. Output hoàn chỉnh 

Ok, tôi sẽ giải thích chi tiết từng phần:

**1. INPUT:**
- Folder ID từ Google Drive (chứa 2 thư mục con: Audio và Video)
```python
folder_id = "1_oVkhMaU4M1-ZIkb5sL4XSRE1LmZUn3z"
```

Cấu trúc folder:
```
Main Folder/
├── Audio/
│   ├── video1_HV.wav      # Audio của học viên
│   └── video1_mentor.wav  # Audio của mentor
└── Video/
    └── video1.mp4         # File video gốc
```

**2. TẠO TRANSCRIPTION:**
```python
def process_audio(self, audio_path: str, language: str = 'en'):
    # 1. Gửi audio file đến transcribe API
    response = requests.post(
        'http://103.253.20.13:25029/role_assign',
        files={'audio': audio_file},
        data={
            'secret_key': 'codedongian',
            'language': language
        }
    )
```

Sau đó combine 2 transcripts:
```python
def combine_transcriptions(self, hv_output: Dict, mentor_output: Dict):
    # 1. Extract text từ output
    hv_data = self.extract_dicts(hv_output['output'])
    mentor_data = self.extract_dicts(mentor_output['output'])

    # 2. Đánh dấu speaker
    for item in hv_data:
        item['speaker'] = '1'
    for item in mentor_data:
        item['speaker'] = '2'

    # 3. Sort theo thời gian và format
    combined = sorted(hv_data + mentor_data, key=lambda x: x['start_time'])
    formatted = [
        f"[{self.format_time(item['start_time'])}] Speaker {item['speaker']}: {item['text']}"
        for item in combined
    ]

    return "\n".join(formatted)
```

**3. LẤY CRITERIA:**
```python
def analyze_transcript(self, transcript: str):
    # Gửi transcript đến analyze API
    response = requests.post(
        'http://103.253.20.13:25034/analyze',
        headers={'Content-Type': 'application/json'},
        json={'transcription': transcript}
    )
    return response.json().get('criteria', '')
```

**4. OUTPUT:**
Lưu vào database `videos` với các trường:
```sql
CREATE TABLE videos (
    id INTEGER PRIMARY KEY,
    url_video TEXT NOT NULL UNIQUE,  -- URL video từ Google Drive
    transcription TEXT,              -- Transcription đã combine
    criteria TEXT,                   -- Kết quả phân tích
    created_at TIMESTAMP            -- Thời điểm xử lý
)
```

**Ví dụ một record hoàn chỉnh:**
```python
{
    'id': 1,
    'url_video': 'https://drive.google.com/file/d/123abc/view',
    'transcription': """
        [00:00:15] Speaker 1: Hello, how are you?
        [00:00:18] Speaker 2: I'm fine, thank you.
        ...
    """,
    'criteria': '<Kết quả phân tích từ API>',
    'created_at': '2024-11-05 10:30:00'
}
```

**Quy trình tổng thể:**
1. Từ video trong folder Video, tìm 2 file audio tương ứng trong folder Audio
2. Process từng file audio qua API role_assign để lấy transcription
3. Combine 2 transcription, sắp xếp theo thời gian
4. Gửi transcript đã combine qua API analyze để lấy criteria
5. Lưu URL video + transcript + criteria vào database

Bạn cần làm rõ phần nào thêm không?

# 6. Update đường dẫn 

Quản lý đường dẫn tập trung:

```python
BASE_DIR = os.path.dirname(os.path.abspath(__file__))
```
Mọi đường dẫn đều relative to BASE_DIR
Dễ di chuyển/deploy code
Tránh lỗi đường dẫn

# 7. UPDATE LOGGING CHI TIẾT HƠN 

Log cho thấy chương trình đã thực hiện thành công các bước đầu tiên:

1. **Database initialized:** 
```
2024-11-05 20:57:49,800 - INFO - Database initialized
```
- Database đã được tạo/kết nối thành công

2. **Looking for folders:**
```
2024-11-05 20:57:49,807 - INFO - Looking for Audio and Video folders in 1_oVkhMaU4M1-ZIkb5sL4XSRE1LmZUn3z
```
- Bắt đầu tìm kiếm trong folder gốc với ID: `1_oVkhMaU4M1-ZIkb5sL4XSRE1LmZUn3z`

3. **Found both folders:**
```
2024-11-05 20:57:50,708 - INFO - Found Audio folder: 1toZNe0bWvgXK5AXjm9uJnL8iDSkcmJjl
2024-11-05 20:57:50,709 - INFO - Found Video folder: 12OTFQ6EHWnfJtfFa8gNaaMzinxsz8eRP
```
- Tìm thấy cả 2 folder Audio và Video
- Mỗi folder có ID riêng để xử lý tiếp

Thêm logging chi tiết hơn để theo dõi các bước tiếp theo:

```python
def process_folder(self, folder_id: str):
    try:
        # Find folders
        audio_folder, video_folder = self.find_folders(folder_id)
        if not audio_folder or not video_folder:
            self.logger.error("Could not find both Audio and Video folders")
            return

        # Get list of videos
        self.logger.info(f"Getting videos from folder: {video_folder}")
        videos = self.drive_service.files().list(
            q=f"'{video_folder}' in parents",
            fields="files(id, name)"
        ).execute().get('files', [])
        self.logger.info(f"Found {len(videos)} videos")

        # Process each video
        for video in videos:
            self.logger.info(f"\nProcessing video: {video['name']}")
            video_url = f"https://drive.google.com/file/d/{video['id']}/view"
            
            # Check if already processed
            if self.db.url_exists(video_url):
                self.logger.info(f"Skipping existing video: {video['name']}")
                continue

            # Find corresponding audio files
            self.logger.info(f"Looking for audio files for video: {video['name']}")
            base_name = os.path.splitext(video['name'])[0]
            audio_files = self.drive_service.files().list(
                q=f"'{audio_folder}' in parents and (name contains '{base_name}')",
                fields="files(id, name)"
            ).execute().get('files', [])
            self.logger.info(f"Found {len(audio_files)} related audio files")

            # Find HV and mentor files
            hv_file = None
            mentor_file = None
            for file in audio_files:
                self.logger.info(f"Checking audio file: {file['name']}")
                if 'HV' in file['name']:
                    hv_file = file
                    self.logger.info(f"Found HV file: {file['name']}")
                elif 'mentor' in file['name']:
                    mentor_file = file
                    self.logger.info(f"Found mentor file: {file['name']}")

            if not (hv_file and mentor_file):
                self.logger.error(f"Missing audio files for video: {video['name']}")
                continue

            try:
                # Download and process HV file
                self.logger.info(f"Processing HV file: {hv_file['name']}")
                hv_path = os.path.join(self.config.TEMP_DIR, f"hv_{video['id']}.wav")
                self.logger.info(f"Downloading HV file to: {hv_path}")
                gdown.download(f"https://drive.google.com/uc?id={hv_file['id']}", 
                             hv_path, quiet=False)
                hv_result = self.audio_processor.process_audio(hv_path)

                # Download and process mentor file
                self.logger.info(f"Processing mentor file: {mentor_file['name']}")
                mentor_path = os.path.join(self.config.TEMP_DIR, f"mentor_{video['id']}.wav")
                self.logger.info(f"Downloading mentor file to: {mentor_path}")
                gdown.download(f"https://drive.google.com/uc?id={mentor_file['id']}", 
                             mentor_path, quiet=False)
                mentor_result = self.audio_processor.process_audio(mentor_path)

                if hv_result and mentor_result:
                    self.logger.info("Combining transcriptions...")
                    transcript = self.audio_processor.combine_transcriptions(
                        hv_result, mentor_result)
                    
                    self.logger.info("Getting criteria...")
                    criteria = self.audio_processor.analyze_transcript(transcript)

                    # Save to database
                    self.logger.info("Saving to database...")
                    if self.db.insert_video(video_url, transcript, criteria):
                        self.logger.info(f"Successfully processed video: {video['name']}")

                # Cleanup
                self.logger.info("Cleaning up temporary files...")
                for path in [hv_path, mentor_path]:
                    if os.path.exists(path):
                        os.remove(path)
                        self.logger.info(f"Removed temporary file: {path}")

            except Exception as e:
                self.logger.error(f"Error processing video {video['name']}: {str(e)}")
                continue

            time.sleep(3)

    except Exception as e:
        self.logger.error(f"Error in main processing: {str(e)}")

```

Sau khi thêm logging chi tiết, bạn có thể theo dõi từng bước:
1. Tìm folders ✓
2. Lấy danh sách videos
3. Với mỗi video:
   - Tìm audio files
   - Download và xử lý HV file
   - Download và xử lý mentor file
   - Combine transcriptions
   - Get criteria
   - Save to database
   - Cleanup

Bạn chạy lại code và cho mình biết các bước tiếp theo có lỗi gì không nhé!

# 8. BUG LƯU DICT JSON VÀO DATABASE

Có 2 lỗi chính đang xảy ra:

1. **Lỗi khi lưu vào database:**
```
ERROR - Database error: Error binding parameter 3: type 'dict' is not supported
```
Nguyên nhân: API analyze trả về một dictionary (dạng JSON), nhưng SQLite không thể lưu trực tiếp kiểu dữ liệu dictionary. 

```python
# Response từ API có dạng:
{
    "criteria": {
        "3Ps": {
            "recommendationScore": {
                "reason": "<b>Present: 8/10</b> Nhận xét: Presentation of ideas was clear..."
            }
        }
    }
}
```

2. **Lỗi encoding tiếng Việt:**
```
UnicodeEncodeError: 'charmap' codec can't encode character '\u1ead' in position 202
```
Nguyên nhân: Logger không thể xử lý các ký tự tiếng Việt với encoding mặc định.

Cách sửa:

```python
# 1. Sửa logging để handle tiếng Việt
def setup_logging(config: Config) -> logging.Logger:
    """Setup logging với UTF-8 encoding"""
    os.makedirs(config.LOG_DIR, exist_ok=True)
    logger = logging.getLogger(__name__)
    logger.setLevel(logging.INFO)
    
    if not logger.handlers:
        # File handler với encoding UTF-8
        fh = logging.FileHandler(
            os.path.join(config.LOG_DIR, 'app.log'),
            encoding='utf-8'
        )
        fh.setFormatter(logging.Formatter('%(asctime)s - %(levelname)s - %(message)s'))
        logger.addHandler(fh)
        
        # Console handler với encoding UTF-8
        ch = logging.StreamHandler()
        ch.setFormatter(logging.Formatter('%(asctime)s - %(levelname)s - %(message)s'))
        logger.addHandler(ch)
    
    return logger

# 2. Sửa hàm analyze_transcript để convert dictionary thành string
def analyze_transcript(self, transcript: str) -> Optional[str]:
    """Analyze transcript và convert kết quả thành string"""
    try:
        self.logger.info("Preparing to analyze transcript...")
        
        response = requests.post(
            self.config.ANALYZE_API_URL,
            headers={'Content-Type': 'application/json'},
            json={"transcription": transcript},
            timeout=60
        )
        
        if response.status_code == 200:
            result = response.json()
            # Convert dictionary thành JSON string để lưu vào DB
            criteria_str = json.dumps(result.get('criteria', {}), 
                                   ensure_ascii=False, 
                                   indent=2)
            return criteria_str
        else:
            self.logger.error(f"API error: {response.status_code}")
            return None

    except Exception as e:
        self.logger.error(f"Error analyzing transcript: {str(e)}")
        return None

# 3. Sửa Database để handle UTF-8
class DatabaseManager:
    def create_database(self):
        """Create database với UTF-8 encoding"""
        with sqlite3.connect(self.db_path) as conn:
            conn.execute('PRAGMA encoding = "UTF-8"')
            conn.execute('''
                CREATE TABLE IF NOT EXISTS videos (
                    id INTEGER PRIMARY KEY,
                    url_video TEXT NOT NULL UNIQUE,
                    transcription TEXT,
                    criteria TEXT,  -- Lưu JSON string
                    created_at TIMESTAMP DEFAULT CURRENT_TIMESTAMP
                )
            ''')
            self.logger.info("Database initialized with UTF-8 encoding")

```

Những thay đổi chính:

1. **Xử lý tiếng Việt:**
- Thêm UTF-8 encoding cho logging
- Set PRAGMA encoding = "UTF-8" cho SQLite
- Dùng `ensure_ascii=False` khi convert JSON

2. **Xử lý dữ liệu criteria:**
- Convert dictionary thành JSON string trước khi lưu
- Khi đọc lại, có thể parse string thành dictionary:
```python
criteria_dict = json.loads(row['criteria'])
```

3. **Cách đọc criteria từ DB:**
```python
def get_video_details(self, video_id: int):
    with self.get_connection() as conn:
        cursor = conn.execute('''
            SELECT id, url_video, transcription, criteria
            FROM videos WHERE id = ?
        ''', (video_id,))
        row = cursor.fetchone()
        if row:
            # Parse criteria string back to dictionary
            criteria = json.loads(row[3]) if row[3] else {}
            return {
                'id': row[0],
                'url': row[1],
                'transcription': row[2],
                'criteria': criteria
            }
        return None
```

Bạn thử chạy lại với những thay đổi này xem còn lỗi không nhé!